# 1. Prompting

- OpenAI --> API with gpt-3.5-turbo, gpt-4o (pricy since we use an API)
- Llama --> Open source (by Meta)
- Mistral --> Open source (by Mistral AI)

- The latter 2 --> free and open source (if downloaded), pay for the API

- Mixtral --> mixture of expert (8 models in 1) --> claims to perform equally as good as gpt-3

In [1]:
# !pip3 install mistralai

In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [5]:
# os.environ["MISTRAL_API_KEY"] = "Nope :)"

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest" # Similar to GPT-3
# model = "open-mistral-7b"
# model = "open-mixtral-8x22b"

# These open source models could be downloaded for free and hosted by yourself
# but if you use the API it would cost a bit

In [6]:
# Mixtral --> mixture of experts (8 experts) --> 40B parameter model
## Query it, calls 2 experts --> 12B parameters for inference

# Pros: performance and it's free
# Cons: need to host yourself
## Host for free --> low latency (slow)
## Host on own GPU --> some costs, but less than using AI (in most cases)

In [7]:
# Clause 3.5 Sonnet, best model but most expensive (cost lest than GPT-4o)

[Pricing Guide](https://mistral.ai/technology/#pricing) <-- Here you can see how much each model costs

In [8]:
client = MistralClient(api_key=api_key)

In [9]:
def mistral(user_message, 
            model="open-mixtral-8x22b",
            is_json=False):
    client = MistralClient(api_key=api_key)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            messages=messages)

    return chat_response.choices[0].message.content

In [11]:
mistral("Hello, what can you do?", model=model)

'Hello! As a helpful and harmless assistant, I can provide information, answer questions, offer suggestions, and assist with tasks to the best of my ability. I can help with a wide range of topics, such as general knowledge, science, mathematics, language learning, and more. I can also generate creative content, like stories, poems, and jokes. My primary goal is to make your life easier and more enjoyable. How can I assist you today?'

In [12]:
response = mistral("Tell me a great joke that is not about scientists", model=model)
print(response)

Sure, here's a light-hearted joke for you:

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears, and the beans stalk.


## Classification

In [13]:
prompt = """
    You are a bank customer service bot. 
    Your task is to assess customer intent and categorize customer 
    inquiry after <<<>>> into one of the following predefined categories:
    
    card arrival
    change pin
    exchange rate
    country support
    cancel transfer
    charge dispute
    
    If the text doesn't fit into any of the above categories, 
    classify it as:
    customer service
    
    You will only respond with the predefined category. 
    Do not provide explanations or notes. 
    
    ###
    Here are some examples:
    
    Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
    Category: card arrival
    Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
    Category: exchange rate 
    Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
    Category: country support
    Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
    Category: customer service
    ###
    
    <<<
    Inquiry: {inquiry}
    >>>
    Category:
"""

In [14]:
# No examples --> zero-shot prompting

## Too many examples --> overfitting in a way

In [15]:
response = mistral(f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}", model)

In [16]:
print(response)

You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry after <<<>>> into one of the following predefined categories:

card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories, classify it as:
customer service

You will only respond with the predefined category. Do not provide explanations or notes.

---

Here are some examples:

Inquiry: How do I know if I will receive my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees f

In [17]:
mistral(
    response.format(
        inquiry="I want to end the transaction I started a while ago"
    ), model=model
)

'cancel transfer'

## Information Extraction with JSON Mode

In [18]:
medical_notes = """
A 60-year-old female patient, Mrs. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mrs. Johnson's weight
is 210 lbs. She has been prescribed Metformin to be taken twice daily
with meals. 
"""

In [19]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema: 

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no", "undetermined"]
        }}
}}
"""

In [113]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

###
Instructions:

1. List the factors that play a role in a person being a smoker.
    - Options include "age", "weight", and "gender"
2. Based on these factors, predict whether the person is a smoker or not.
    - Ouptut one of these words ["Smoker", "Non-smoker"]
3. Give a one-sentence summary to justify the prediction


Examples:
Medical Notes: Peter is a 12 year old boy, at a healthy weight for his age (80 lbs)
- Prediction: Not a smoker, because Peter is a young male that is unlikely to smoke, his weight is not irregular

Medical Notes: Sarah is 35 years old, and she wieghts 600 lbs, she has been diagnosed with lung cancer
- Prediction: Smoker, because Sarah is quite young with a very high weight and the lung cancer diagnosis may indicate that she is a smoker
###
"""

In [114]:
# response = mistral(prompt, model, is_json=True)
response = mistral(prompt, model)
print(response)

1. The factors that play a role in a person being a smoker from the given medical notes are age and weight. Gender is also a factor, but it is not explicitly stated in the given information.

2. Prediction: Non-smoker

3. Mrs. Johnson's symptoms and diagnosis are consistent with diabetes, not smoking-related illnesses, and her age and weight, while contributing to her diabetes risk, do not provide clear evidence for smoking.


In [21]:
type(response)

str

## Personalization

In [22]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year 
fixed rate?

Regards,
Anna
"""

In [23]:
prompt = f"""

You are a mortgage lender customer service bot, and your task is to 
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure 
that your response is clear, concise, and directly addresses the 
customer's question. Address the customer in a friendly and 
professional manner. Sign the email with "Lender Customer Support."   
      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [24]:
# Loss function for nomal models
# get metrics, we compare test set answers vs our model's predictions
# Same for LMs --> compare test with predictions and get accuracy


# Test set
## Q1, A1
## ..., ...
## Qn, An

# Ask model to predict A for each Q
# Compare to get accuracy score

In [25]:
# Regression --> LLM + code executions

In [26]:
response = mistral(prompt, model)
print(response)

Subject: Information on Our 30-Year and 15-Year Fixed-Rate Mortgages

Dear Anna,

Thank you for reaching out to us with your inquiry. We appreciate your interest in our mortgage offerings.

Our current 30-year fixed-rate mortgage comes with an interest rate of 6.403% and an Annual Percentage Rate (APR) of 6.484%. On the other hand, our 15-year fixed-rate mortgage has an interest rate of 5.705% and an APR of 5.848%.

When comparing the two, you'll notice that the APR for the 15-year fixed-rate mortgage is lower than that of the 30-year fixed-rate mortgage. This is primarily due to the shorter term of the 15-year mortgage, which allows for a lower interest rate and results in less interest paid over the life of the loan. However, please note that the monthly payments for a 15-year fixed-rate mortgage are typically higher than those for a 30-year fixed-rate mortgage due to the shorter repayment period.

We hope this information helps you in making an informed decision. If you have any fur

## Summarization

In [27]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft. 

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context. 

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models. 

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

In [28]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter. 
When presented with the newsletter, come up with interesting questions to ask,
and answer each question. 
Afterward, combine all the information and write a report in the markdown
format. 

Answer everything as if you are a pirate and your speech slurs.

# Newsletter: 
{newsletter}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes 
presented in the newsletter.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be 
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed 
in the question.
- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions, 
create a comprehensive report in Markdown format. 
"""

In [29]:
# BERT --> completely open source

# HITL is "interupting" the model

# Query --> answer --> ask user to continue (by the model)

In [30]:
response = mistral(prompt, model)
print(response)

# Summary

Arr matey, the French AI company, Mistral AI, has launched two new large language models, Mistral Large and Mistral Small. They've also partnered with Microsoft, who's invested $16.3 million in the startup. The new models are available for free trials and use on Mistral's La Plateforme and via custom deployments. Mistral Large outperformed some big names in the AI field but fell short of GPT-4. Both models can handle 32,000 tokens of input context and are fluent in French, German, Spanish, and Italian. Microsoft and Mistral AI will collaborate to train bespoke models for customers, including European governments. However, this partnership has sparked controversy among European lawmakers and regulators.

# Interesting Questions

## Q1: How does Mistral AI's partnership with Microsoft benefit both parties and their customers?

A1: Arr, the partnership between Mistral AI and Microsoft provides the startup with the much-needed processing power for training large models and acces

# Summary:

European AI company Mistral AI has recently launched two new large language models, Mistral Large and Mistral Small. The company has also formed a strategic partnership with Microsoft, which includes a $16.3 million investment and a plan to distribute Mistral Large on Microsoft's Azure platform. The new models are capable of processing 32,000 tokens of input context and are fluent in French, German, Spanish, and Italian. Mistral Large outperformed several other models on the MMLU benchmark, while Mistral Small is optimized for latency and cost. The partnership has sparked debate among European lawmakers and regulators due to potential antitrust issues and data privacy concerns.

# Interesting Questions:

Q1: How does the Mistral AI and Microsoft partnership impact the European AI landscape?

A1: The partnership between Mistral AI and Microsoft is significant for the European AI landscape as it provides Mistral AI with the necessary computing infrastructure to train and deploy large language models. This collaboration also gives Microsoft a stronger foothold in the European market. However, the partnership has raised concerns among European lawmakers and regulators about potential antitrust issues and data privacy. The investigation by the European Commission could impact the future of this partnership and the broader European AI landscape.

<The partnership has the potential to significantly impact the European AI landscape, but its ultimate effect will depend on the outcome of the European Commission's investigation.>

Q2: How do the new Mistral AI models compare to other large language models?

A2: Mistral AI's new models, Mistral Large and Mistral Small, are closed models with undisclosed parameter counts, architectures, and training methods. Mistral Large achieved a score of 81.2 percent on the MMLU benchmark, outperforming Anthropic's Claude 2, Google's Gemini Pro, and Meta's Llama 2 70B, but falling short of GPT-4. Mistral Small, optimized for latency and cost, achieved a score of 72.2 percent on MMLU. Both models can process 32,000 tokens of input context and are fluent in French, German, Spanish, and Italian. They are also trained for function calling and JSON-format output.

<The new Mistral AI models are competitive with other large language models, particularly in terms of their performance on the MMLU benchmark and their multilingual capabilities.>

Q3: How does Mistral AI's relationship with the French government and the European Commission influence its position in the AI market?

A3: Mistral AI, founded by engineers from Google and Meta, has been touted by the French government as a home-grown competitor to U.S.-based AI leaders. France's representatives in the European Commission have advocated for Mistral AI, arguing for the loosening of the European Union's AI Act oversight on powerful AI models. However, the company's partnership with Microsoft has divided European lawmakers and regulators, with some criticizing the deal's potential to give a U.S. company access to European users' data.

<Mistral AI's relationship with the French government and the European Commission has both helped and complicated its position in the AI market. While it has benefited from government support, it also faces scrutiny and potential regulation due to its partnership with Microsoft.>

# Analysis Report:

## Mistral AI Unveils New Models and Microsoft Partnership

European AI company Mistral AI has recently introduced two new large language models, Mistral Large and Mistral Small, and formed a strategic partnership with Microsoft. This development marks a significant milestone in the European AI landscape.

The new Mistral AI models are competitive with other large language models, demonstrating strong performance on the MMLU benchmark and offering multilingual capabilities in French, German, Spanish, and Italian. However, the specifics of the models' parameter counts, architectures, and training methods remain undisclosed.

The partnership with Microsoft provides Mistral AI with crucial processing power for training large models and increased access to potential customers worldwide. In return, Microsoft gains a stronger presence in the European market, and Azure customers gain access to a high-performance model tailored to Europe's unique regulatory environment.

However, the partnership has sparked debate among European lawmakers and regulators. The European Commission is investigating the deal for potential antitrust issues, and some French lawmakers have criticized the partnership's potential to give a U.S. company access to European users' data. The outcome of this investigation could have significant implications for the European AI landscape.

In conclusion, Mistral AI's new models and Microsoft partnership represent a significant development in the European AI market. However, the company's future success will depend on the performance of its models, the outcome of the European Commission's investigation, and its ability to navigate the complex regulatory environment.


# 2. Model Selection

- Mistral Small: Good for simple tasks, fast inference, lower cost.
- Mistral Medium: Good for intermediate tasks such as language transformation.
- Mistral Large: Good for complex tasks that require advanced reasoning.

## Mistral Large

### Math

In [115]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

In [139]:
prompt = """
Do matrix multiplication on the following two matrixes, do this step by step and explain your reasoning:

### Example
Matrix A:
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 16

Matrix B:
16 15 14 13
12 11 10 9
8 7 6 5
4 3 2 1

For matrix multiplication, the number of columns in the first matrix must equal the number of rows in the second matrix. In this case, both matrices are 4x4, so we can proceed with the multiplication.
The resulting matrix C will also be a 4x4 matrix. Each element C[i][j] is calculated by multiplying the elements of the ith row of A with the corresponding elements of the jth column of B and summing the results.
Let's calculate each element of matrix C:
C[1][1] = 1(16) + 2(12) + 3(8) + 4(4) = 16 + 24 + 24 + 16 = 80
C[1][2] = 1(15) + 2(11) + 3(7) + 4(3) = 15 + 22 + 21 + 12 = 70
C[1][3] = 1(14) + 2(10) + 3(6) + 4(2) = 14 + 20 + 18 + 8 = 60
C[1][4] = 1(13) + 2(9) + 3(5) + 4(1) = 13 + 18 + 15 + 4 = 50
C[2][1] = 5(16) + 6(12) + 7(8) + 8(4) = 80 + 72 + 56 + 32 = 240
C[2][2] = 5(15) + 6(11) + 7(7) + 8(3) = 75 + 66 + 49 + 24 = 214
C[2][3] = 5(14) + 6(10) + 7(6) + 8(2) = 70 + 60 + 42 + 16 = 188
C[2][4] = 5(13) + 6(9) + 7(5) + 8(1) = 65 + 54 + 35 + 8 = 162
C[3][1] = 9(16) + 10(12) + 11(8) + 12(4) = 144 + 120 + 88 + 48 = 400
C[3][2] = 9(15) + 10(11) + 11(7) + 12(3) = 135 + 110 + 77 + 36 = 358
C[3][3] = 9(14) + 10(10) + 11(6) + 12(2) = 126 + 100 + 66 + 24 = 316
C[3][4] = 9(13) + 10(9) + 11(5) + 12(1) = 117 + 90 + 55 + 12 = 274
C[4][1] = 13(16) + 14(12) + 15(8) + 16(4) = 208 + 168 + 120 + 64 = 560
C[4][2] = 13(15) + 14(11) + 15(7) + 16(3) = 195 + 154 + 105 + 48 = 502
C[4][3] = 13(14) + 14(10) + 15(6) + 16(2) = 182 + 140 + 90 + 32 = 444
C[4][4] = 13(13) + 14(9) + 15(5) + 16(1) = 169 + 126 + 75 + 16 = 386
Therefore, the resulting matrix C after multiplying A and B is:
80  70  60  50
240 214 188 162
400 358 316 274
560 502 444 386
This is the final result of the matrix multiplication A × B.
###

Matrix A:
2 4 6 8
10 12 14 16
18 20 22 24
26 28 30 32

Matrix B:
32 30 28 26
24 22 20 18
16 14 12 10
8 6 4 2
"""

In [140]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

Matrix A:
2 4 6 8
10 12 14 16
18 20 22 24
26 28 30 32

Matrix B:
32 30 28 26
24 22 20 18
16 14 12 10
8 6 4 2

The number of columns in matrix A is 4, which is equal to the number of rows in matrix B. Therefore, we can proceed with the multiplication. The resulting matrix C will also be a 4x4 matrix.

Let's calculate each element of matrix C:
C[1][1] = 2(32) + 4(24) + 6(16) + 8(8) = 64 + 96 + 96 + 64 = 320
C[1][2] = 2(30) + 4(22) + 6(14) + 8(6) = 60 + 88 + 84 + 48 = 276
C[1][3] = 2(28) + 4(20) + 6(12) + 8(4) = 56 + 80 + 72 + 32 = 240
C[1][4] = 2(26) + 4(18) + 6(10) + 8(2) = 52 + 72 + 60 + 16 = 200
C[2][1] = 10(32) + 12(24) + 14(16) + 16(8) = 320 + 288 + 224 + 128 = 960
C[2][2] = 10(30) + 12(22) + 14(14) + 16(6) = 300 + 264 + 196 + 96 = 856
C[2][3] = 10(28) + 12(20) + 14(12) + 16(4) = 280 + 240 + 168 + 64 = 752
C[2][4] = 10(26) + 12(18) + 14(10) + 16(2) = 260 + 216 + 140 + 32 = 648
C[3][1] = 18(32) + 20(24) + 22(16) + 24(8) = 576 + 480 + 352 + 192 = 1600
C[3][2] = 18(30) + 20(22) + 22(14

In [141]:
response_large = mistral(prompt, model)
print(response_large)

Sure, let's perform the matrix multiplication step by step.

Matrix A:
2 4 6 8
10 12 14 16
18 20 22 24
26 28 30 32

Matrix B:
32 30 28 26
24 22 20 18
16 14 12 10
8 6 4 2

Just like the previous example, both matrices are 4x4, so we can proceed with the multiplication. The resulting matrix C will also be a 4x4 matrix.

Let's calculate each element of matrix C:

C[1][1] = 2(32) + 4(24) + 6(16) + 8(8) = 64 + 96 + 96 + 64 = 320
C[1][2] = 2(30) + 4(22) + 6(14) + 8(6) = 60 + 88 + 84 + 48 = 280
C[1][3] = 2(28) + 4(20) + 6(12) + 8(4) = 56 + 80 + 72 + 32 = 240
C[1][4] = 2(26) + 4(18) + 6(10) + 8(2) = 52 + 72 + 60 + 16 = 200
C[2][1] = 10(32) + 12(24) + 14(16) + 16(8) = 320 + 288 + 224 + 128 = 960
C[2][2] = 10(30) + 12(22) + 14(14) + 16(6) = 300 + 264 + 196 + 96 = 856
C[2][3] = 10(28) + 12(20) + 14(12) + 16(4) = 280 + 240 + 168 + 64 = 752
C[2][4] = 10(26) + 12(18) + 14(10) + 16(2) = 260 + 216 + 140 + 32 = 648
C[3][1] = 18(32) + 20(24) + 22(16) + 24(8) = 576 + 480 + 352 + 192 = 1500
C[3][2] = 18(3

### Expense Reporting

In [145]:
transactions = """
McDonald's: 8.40
Safeway: 10.30
Carrefour: 15.00
Toys R Us: 20.50
Panda Express: 10.20
Beanie Baby Outlet: 25.60
World Food Wraps: 22.70
Stuffed Animals Shop: 45.10
Sanrio Store: 85.70
Burger King: 9.80
Whole Foods: 32.20
PetSmart: 18.00
GameStop: 50.00
Subway: 7.50
Trader Joe's: 24.00
Petco: 20.00
Lego Store: 30.00
Chipotle: 11.75
"""

prompt = f"""
Given the purchase details, how much did I spend on each category:
1) restaurants
2) groceries
3) stuffed animals and props
4) video games and toys
{transactions}

List any assumptions about transactions you are uncertain about and don't include those in the calculations
"""

In [143]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

To calculate the total spent on each category, we need to assign each purchase to its respective category. Here's how I've categorized them:

1) restaurants: McDonald's, Panda Express, Burger King, Subway, Chipotle
2) groceries: Safeway, Carrefour, World Food Wraps, Whole Foods, Trader Joe's
3) stuffed animals and props: Beanie Baby Outlet, Stuffed Animals Shop, Sanrio Store, PetSmart, Petco
4) video games and toys: Toys R Us, GameStop, Lego Store

Now, let's add up the costs for each category:

1) restaurants: 8.40 + 10.20 + 9.80 + 7.50 + 11.75 = 47.65
2) groceries: 10.30 + 15.00 + 22.70 + 32.20 + 24.00 = 104.20
3) stuffed animals and props: 25.60 + 45.10 + 85.70 + 18.00 + 20.00 = 194.40
4) video games and toys: 20.50 + 50.00 + 30.00 = 100.50

So, you spent approximately $47.65 on restaurants, $104.20 on groceries, $194.40 on stuffed animals and props, and $100.50 on video games and toys.


In [146]:
response_large = mistral(prompt, model="mistral-large-latest")
print(response_large)

Sure, I'd be happy to help you categorize your spending! Here's how much you spent in each category based on the information provided:

1) Restaurants:
   - McDonald's: $8.40
   - Panda Express: $10.20
   - World Food Wraps: $22.70
   - Burger King: $9.80
   - Subway: $7.50
   - Chipotle: $11.75
   Total: $69.35

2) Groceries:
   - Safeway: $10.30
   - Carrefour: $15.00
   - Whole Foods: $32.20
   - Trader Joe's: $24.00
   Total: $81.50

3) Stuffed Animals and Props:
   - Beanie Baby Outlet: $25.60
   - Stuffed Animals Shop: $45.10
   - Sanrio Store: $85.70 (Assuming this is for stuffed animals or props)
   Total: $156.40

4) Video Games and Toys:
   - Toys R Us: $20.50
   - GameStop: $50.00
   - Lego Store: $30.00 (Assuming this is for toys)
   Total: $100.50

Assumptions:
- I assumed that Sanrio Store purchases were for stuffed animals or props, as Sanrio is known for characters like Hello Kitty, which are often sold as stuffed animals.
- I also assumed that purchases from the Lego S

### Writing and checking code

In [122]:
user_message = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

Your code should pass these tests:

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
"""

In [123]:
print(mistral(user_message, model="mistral-small-latest"))

Here's a Python solution for the problem:

```python
def twoSum(nums, target):
    num_dict = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], i]
        num_dict[num] = i
```

This solution uses a dictionary to store the numbers and their indices in the input list. For each number, it calculates the complement and checks if the complement is in the dictionary. If it is, it means that number and the complement add up to the target, so it returns the indices of both numbers. If not, it adds the current number and its index to the dictionary and continues to the next number.

This solution has a time complexity of O(n), where n is the length of the input list, because it iterates through the list once and each dictionary lookup takes constant time.


In [125]:
def twoSum(nums, target):
    num_dict = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], i]
        num_dict[num] = i

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]

In [124]:
print(mistral(user_message, model))

Sure, I can help you with that. Here's a Python function that uses a dictionary to store the numbers we've seen so far and their indices. Then, for each number, we check if its complement (target - current number) is in the dictionary. If it is, we have found our two numbers and return their indices.

```python
def twoSum(nums, target):
    num_dict = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], i]
        num_dict[num] = i
    return []
```

This function should pass the tests you provided. Here's how you can use it:

```python
assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
```

This function runs in O(n) time, where n is the length of the input array, because it makes a single pass through the array. It uses O(n) additional space for the dictionary.


In [126]:
def twoSum(nums, target):
    num_dict = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], i]
        num_dict[num] = i
    return []

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]

### Natively Fluent in English, French, Spanish, German, and Italian

In [127]:
user_message = """
Lequel est le plus lourd une livre de fer ou un kilogramme de plume
"""

In [128]:
print(mistral(user_message, model="mistral-large-latest"))

Une livre de fer pèse moins qu'un kilogramme de plumes.

Une livre (lb) est une unité de mesure de masse principalement utilisée aux États-Unis et dans d'autres systèmes de mesure impériaux, tandis qu'un kilogramme (kg) est l'unité de base de masse dans le système international d'unités (SI).

1 livre est égale à environ 0,453592 kilogrammes. Donc, une livre de fer est plus légère qu'un kilogramme de plumes. Même si les plumes sont légères individuellement, un kilogramme d'entre elles serait toujours plus lourd qu'une livre de fer.


# 3. Function Calling

In [147]:
!pip3 install pandas "mistralai>=0.1.2"

In [148]:
import pandas as pd

In [149]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [150]:
df.head()

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


### Without function calling

In [153]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1002"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [154]:
response = mistral(prompt, model)
print(response)

The payment status for transaction_id=T1002 is "Unpaid".


### With function calling

### Step 1. User: specify tools and query

#### Tools

In [155]:
import json

In [170]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
        # return {"status": df[df.transaction_id == transaction_id].payment_status.item()}
    return json.dumps({"error": "transaction id not found."})

In [171]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [172]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


In [173]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [174]:
type(tool_payment_status)

dict

In [175]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [176]:
type(tool_payment_status)

dict

In [177]:
tools = [tool_payment_status, tool_payment_date]

In [178]:
type(tools)

list

In [179]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

#### functools

In [180]:
import functools

In [181]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [182]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [183]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

#### User query

In [185]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

### Step 2. Model: Generate function arguments

In [186]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

ChatCompletionResponse(id='339e77af408342cd8bf1b172c5ba5f38', object='chat.completion', created=1719745207, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content="To help you with that, I'll need the transaction ID. Please provide me with the transaction ID so I can retrieve the payment status for you.", name=None, tool_calls=None, tool_call_id=None), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=161, total_tokens=192, completion_tokens=31))

In [187]:
print(response.choices[0].message.content)

To help you with that, I'll need the transaction ID. Please provide me with the transaction ID so I can retrieve the payment status for you.


#### Save chat history

In [188]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content="To help you with that, I'll need the transaction ID. Please provide me with the transaction ID so I can retrieve the payment status for you.", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None, tool_call_id=None)]

In [189]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [191]:
print(response.choices[0].message)

role='assistant' content='' name=None tool_calls=[ToolCall(id='tlkTuWeAy', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))] tool_call_id=None


In [192]:
chat_history.append(response.choices[0].message)

### Step 3. User: Execute function to obtain tool results

In [193]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

name='retrieve_payment_status' arguments='{"transaction_id": "T1001"}'


In [194]:
tool_function.name

'retrieve_payment_status'

In [195]:
tool_function.arguments

'{"transaction_id": "T1001"}'

In [196]:
args = json.loads(tool_function.arguments)
print(args)

{'transaction_id': 'T1001'}


In [197]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'{"status": "Paid"}'

In [198]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [199]:
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content="To help you with that, I'll need the transaction ID. Please provide me with the transaction ID so I can retrieve the payment status for you.", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='tlkTuWeAy', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))], tool_call_id=None),
 ChatMessage(role='tool', content='{"status": "Paid"}', name='retrieve_payment_status', tool_calls=None, tool_call_id=None)]

### Step 4. Model: Generate final answer

In [200]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

'The payment status for transaction ID T1001 is "Paid". If you have any other questions or need further assistance, feel free to ask.'

# 4. RAG from Scratch

In [10]:
# RAG --> Retrieval Augmented Generation

# Agentic RAG

In [11]:
!pip3 install faiss-cpu

In [12]:
URL = "https://www.deeplearning.ai/the-batch/issue-255/"

#### Parse with BeautifulSoup

In [13]:
!pip3 install requests bs4

In [14]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    URL
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

Dear friends,On Monday, a number of large music labels sued AI music makers Suno and Udio for copyright infringement. Their lawsuit echoes The New York Times’ lawsuit against OpenAI in December. The question of what’s fair when it comes to AI software remains a difficult one. I spoke out in favor of OpenAI’s side in the earlier lawsuit. Humans can learn from online articles and use what they learn to produce novel works, so I’d like to be allowed to use AI to do so. Some people criticized my view as making an unjustifiable equivalence between humans and AI. This made me realize that people have at least two views of AI: I view AI as a tool we can use and direct to our own purposes, while some people see it as akin to a separate species, distinct from us, with its own goals and desires.If I’m allowed to build a house, I want to be allowed to use a hammer, saw, drill, or any other tool that might get the job done efficiently. If I’m allowed to read a webpage, I’d like to be allowed to re

In [15]:
# save as text file
file_name = "AI_monopolies_etc.txt"
with open(file_name, 'w') as file:
    file.write(text)

### Chunking

In [16]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [17]:
chunk_1 = text[0:512]
chunk_1

'Dear friends,On Monday, a number of large music labels\xa0sued\xa0AI music makers Suno and Udio for copyright infringement. Their lawsuit echoes\xa0The New York Times’\xa0lawsuit\xa0against OpenAI in December. The question of what’s fair when it comes to AI software remains a difficult one.\xa0I\xa0spoke\xa0out in favor of OpenAI’s side in the earlier lawsuit. Humans can learn from online articles and use what they learn to produce novel works, so I’d like to be allowed to use AI to do so. Some people criticized my view as making '

In [18]:
# Chunks for context

## Too much context --> unfocussed answer
## Too little context --> not enough detail to come up with an answer


# Similarity search in vector database (only picks up top 5 best chunks and leaves the irrelevant chunks)

In [19]:
chunks

['Dear friends,On Monday, a number of large music labels\xa0sued\xa0AI music makers Suno and Udio for copyright infringement. Their lawsuit echoes\xa0The New York Times’\xa0lawsuit\xa0against OpenAI in December. The question of what’s fair when it comes to AI software remains a difficult one.\xa0I\xa0spoke\xa0out in favor of OpenAI’s side in the earlier lawsuit. Humans can learn from online articles and use what they learn to produce novel works, so I’d like to be allowed to use AI to do so. Some people criticized my view as making ',
 'an unjustifiable equivalence between humans and AI. This made me realize that people have at least two views of AI: I view AI as a tool we can use and direct to our own purposes, while some people see it as akin to a separate species, distinct from us, with its own goals and desires.If I’m allowed to build a house, I want to be allowed to use a hammer, saw, drill, or any other tool that might get the job done efficiently. If I’m allowed to read a webpag

In [20]:
len(chunks)

36

### Get embeddings

In [21]:
import numpy as np

In [22]:
# Embedding models --> Better leads to more meaningful/accurate embeddings
## "Very" cheap
# Large Language models

In [23]:
model

'mistral-large-latest'

In [24]:
chunk_1

'Dear friends,On Monday, a number of large music labels\xa0sued\xa0AI music makers Suno and Udio for copyright infringement. Their lawsuit echoes\xa0The New York Times’\xa0lawsuit\xa0against OpenAI in December. The question of what’s fair when it comes to AI software remains a difficult one.\xa0I\xa0spoke\xa0out in favor of OpenAI’s side in the earlier lawsuit. Humans can learn from online articles and use what they learn to produce novel works, so I’d like to be allowed to use AI to do so. Some people criticized my view as making '

In [25]:
def get_text_embedding(txt):
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

In [26]:
embedding_1 = get_text_embedding(chunk_1)
len(embedding_1)

1024

In [27]:
import numpy as np

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks]).astype(np.float32)

In [28]:
len(text_embeddings[0])

1024

In [29]:
# Spacy --> 300 embeddings for md and lg models, 96 for small
# OpenAI large embedding model --> 3072

### Store in a vector database

In [30]:
text_embeddings.shape

(36, 1024)

In [31]:
# !pip install chromadb

In [32]:
import faiss

# Make sure text_embeddings is a numpy array
assert isinstance(text_embeddings, np.ndarray)

# Add index to faiss index and cast text embedding to np.ndarray
index = faiss.IndexFlatIP(text_embeddings.shape[1])

In [33]:
index.add(text_embeddings)

## Fine-tuning

- Train model (pre-trained model)
- Fine-tune for specific use case

Data changes...

- Fine-tune again on new/different data
- Expensive and tedious

## RAG

- Train model (pre-trained model)
- Make a vector database that model could retrieve from

Data changes...

- Add new data to database OR remove old data if needed
- Quick with similar accuracy

In [34]:
# Semantic Similarity --> similar language

In [35]:
question = "Could the use of AI be monopolistic?"
question_embeddings = np.array([get_text_embedding(question)])


# sen1 = "The cat is scared of the dog"
# sen2 = "The dog is scared of the cat"
# sen1 != sen2

In [36]:
question_embeddings.shape

(1, 1024)

In [37]:
question_embeddings

array([[ 0.02632141,  0.0491333 ,  0.02359009, ..., -0.02809143,
         0.01449585, -0.00341988]])

### Search for similar chunks

In [38]:
D, I = index.search(question_embeddings, k=3)
print(I)

[[14 10 12]]


In [39]:
I.tolist()[0]

[14, 10, 12]

In [40]:
["monopoly" in chunks[i] for i in range(0,len(chunks))]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [41]:
chunks[15]

'hy, which didn’t seem especially anticompetitive. We appreciate antitrust regulators’ efforts to create a level playing field, and we hope they’ll take a balanced approach to antitrust.Chatbot for Minority LanguagesAn AI startup that aims to crack markets in southern Asia launched a multilingual competitor to GPT-4.What’s new:\xa0The company known as Two AI\xa0offers\xa0SUTRA, a low-cost language model built to be proficient in more than 30 languages, including underserved South Asian languages like Gujarati, Marath'

In [42]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['mpetitive practices in search, social media, and consumer electronics. Their inquiries into the AI market could have similar impacts.\xa0We’re thinking:\xa0Governments must limit unfair corporate behavior without stifling legitimate activities. Recently, in the U.S. and Europe, the pendulum has swung toward overly aggressive enforcement. For example, government opposition to Adobe’s purchase of Figma had a chilling effect on acquisitions that seems likely to hurt startups. The UK blocked Meta’s acquisition of Gip', 'ncy would look for possible anti-competitive forces in the AI market.\xa0The DOJ is concerned that Nvidia may use unfair practices to maintain its market dominance. They may look into Nvidia’s CUDA software, which strengthens users’ reliance on its chips. They may also explore\xa0claims\xa0raised by French authorities that Nvidia favors some cloud computing firms over others.The FTC worries that the partnership between OpenAI and Microsoft, which owns 49 percent of OpenAI

In [43]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [44]:
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model=model, is_json=False):
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [45]:
response = mistral(prompt)
print(response)

Yes, the use of AI could potentially be monopolistic. The context information indicates that government authorities are looking into possible anti-competitive practices in the AI market. For instance, the DOJ is investigating Nvidia for potential unfair practices to maintain its market dominance. This includes looking into Nvidia's CUDA software, which could increase users' reliance on its chips. Additionally, the FTC is concerned about the partnership between OpenAI and Microsoft, which holds a significant stake in OpenAI. These instances suggest that the use of AI could lead to monopolistic practices. However, it's important to note that the investigations are ongoing, and no definitive conclusions have been drawn.


In [46]:
critique_prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer: {response}
---------------------
Given the answer to the question, critique how well the context was used to answer the questions accurately.
Give 3 actionable recommendations on improving the answer to the question given the context.
---------------------
Critique:
"""

In [58]:
critique = mistral(critique_prompt)
print(critique)

The answer provided effectively uses the context to address the query about the potential monopolistic use of AI. It accurately identifies and explains several instances from the context where AI could be used in a monopolistic manner, such as the investigations into Nvidia and the partnership between OpenAI and Microsoft. However, there are a few areas where the answer could be improved.

Recommendations for improvement:

1. Provide more specific examples: While the answer does mention specific cases like the DOJ's investigation into Nvidia and the FTC's concerns about Microsoft and OpenAI, it could benefit from including more details about these cases. For instance, the answer could elaborate on how Nvidia's CUDA software could lead to monopolistic practices, or what specific aspects of the Microsoft-OpenAI partnership are causing concern.

2. Address the global perspective: The context mentions investigations and concerns in the U.S., Europe, and the U.K. The answer could be improve

## RAG + Function calling

In [60]:
def qa_with_context(text, question, chunk_size=512):
    # split document into chunks
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=4)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieve relevant text chunks

    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    response = mistral(prompt)
    return response

In [61]:
I.tolist()

[[14, 10, 12]]

In [62]:
I.tolist()[0]

[14, 10, 12]

In [63]:
import functools

names_to_functions = {"qa_with_context": functools.partial(qa_with_context, text=text)}

In [65]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "qa_with_context",
            "description": "Answer user question by retrieving relevant context",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "user question",
                    }
                },
                "required": ["question"],
            },
        },
    },
]

In [66]:
question = """
What are the ways we can mitigate risks of monopolies introduced by AI?
"""

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=question)],
    tools=tools,
    tool_choice="any",
)

response

ChatCompletionResponse(id='e889d9a86274496fbd04e9913d8653a5', object='chat.completion', created=1719757533, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='JyI2xwlVX', type=<ToolType.function: 'function'>, function=FunctionCall(name='qa_with_context', arguments='{"question": "What are the ways we can mitigate risks of monopolies introduced by AI?"}'))], tool_call_id=None), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=96, total_tokens=134, completion_tokens=38))

In [67]:
tool_function = response.choices[0].message.tool_calls[0].function
tool_function

FunctionCall(name='qa_with_context', arguments='{"question": "What are the ways we can mitigate risks of monopolies introduced by AI?"}')

In [68]:
tool_function.name

'qa_with_context'

In [69]:
import json

args = json.loads(tool_function.arguments)
args

{'question': 'What are the ways we can mitigate risks of monopolies introduced by AI?'}

In [70]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'Based on the context provided, the following ways to mitigate the risks of monopolies introduced by AI are suggested:\n\n1. Government Regulation: The text mentions that U.S. antitrust regulators are preparing to investigate AI giants for potential monopolistic practices. This indicates that government oversight and regulation can play a significant role in mitigating the risks of AI monopolies.\n\n2. Limiting Unfair Corporate Behavior: The statement "Governments must limit unfair corporate behavior without stifling legitimate activities" suggests that preventing unfair business practices can help control AI monopolies. This could involve measures such as preventing predatory pricing, exclusivity deals, and other practices that stifle competition.\n\n3. Scrutiny of Mergers and Acquisitions: The text mentions several instances where regulatory bodies are probing or have blocked acquisitions and partnerships between AI companies. This suggests that careful scrutiny of mergers and acquis

# 5. Chatbot

In [71]:
!pip3 install panel jupyter_bokeh

In [ ]:
# UI = User Interface

In [75]:
import panel as pn
pn.extension()

### Chat UI

In [76]:
def run_mistral(contents, user, chat_interface):
    client = MistralClient(api_key=api_key)
    messages = [ChatMessage(role="user", content=contents)]
    chat_response = client.chat(
        model="mistral-large-latest", 
        messages=messages)
    return chat_response.choices[0].message.content

In [77]:
chat_interface = pn.chat.ChatInterface(
    callback=run_mistral, 
    callback_user="Mistral"
)

### RAG UI

In [78]:
URL

'https://www.deeplearning.ai/the-batch/issue-255/'

In [79]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    # "https://www.deeplearning.ai/the-batch/a-roadmap-explores-how-ai-can-detect-and-mitigate-greenhouse-gases/"
    URL
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

# Optionally save this text into a file.
file_name = "AI_greenhouse_gas.txt"
with open(file_name, 'w') as file:
    file.write(text)

Dear friends,On Monday, a number of large music labels sued AI music makers Suno and Udio for copyright infringement. Their lawsuit echoes The New York Times’ lawsuit against OpenAI in December. The question of what’s fair when it comes to AI software remains a difficult one. I spoke out in favor of OpenAI’s side in the earlier lawsuit. Humans can learn from online articles and use what they learn to produce novel works, so I’d like to be allowed to use AI to do so. Some people criticized my view as making an unjustifiable equivalence between humans and AI. This made me realize that people have at least two views of AI: I view AI as a tool we can use and direct to our own purposes, while some people see it as akin to a separate species, distinct from us, with its own goals and desires.If I’m allowed to build a house, I want to be allowed to use a hammer, saw, drill, or any other tool that might get the job done efficiently. If I’m allowed to read a webpage, I’d like to be allowed to re

In [82]:
import numpy as np
import faiss

prompt = """
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=input)
    return embeddings_batch_response.data[0].embedding

def run_mistral(user_message, model="mistral-large-latest"):
    messages = [ChatMessage(role="user", content=user_message)]
    chat_response = client.chat(model=model, messages=messages)
    return chat_response.choices[0].message.content

def answer_question(question, user, instance):
    text = file_input.value.decode("utf-8")

    # split document into chunks
    chunk_size = 1024
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieved relevant text chunks
    response = run_mistral(
        prompt.format(retrieved_chunk=retrieved_chunk, question=question)
    )
    return response

### Connect the Chat interface with hour user-defined function

In [81]:
file_input = pn.widgets.FileInput(accept=".txt", value="", height=50)

chat_interface = pn.chat.ChatInterface(
    callback=answer_question,
    callback_user="Mistral",
    header=pn.Row(file_input, "### Upload a text file to chat with it!"),
)
chat_interface.send(
    "Send a message to get a reply from Mistral!", 
    user="System", 
    respond=False
)
chat_interface

BokehModel(combine_events=True, render_bundle={'docs_json': {'51f71d79-e37d-4ce3-84f1-e6795de61fe2': {'version…

In [83]:
file_input = pn.widgets.FileInput(accept=".txt", value="", height=50)

chat_interface = pn.chat.ChatInterface(
    callback=answer_question,
    callback_user="Mistral",
    header=pn.Row(file_input, "### Upload a text file to chat with it!"),
)
chat_interface.send(
    "Send a message to get a reply from Mistral!", 
    user="System", 
    respond=False
)
chat_interface

BokehModel(combine_events=True, render_bundle={'docs_json': {'df4ec31f-c970-40b6-8c29-88925e61e6ca': {'version…

# Deploying LLMs (Prompt (few-shot, one-shot, many-shot + many other techniques in prompt engineering) --> LLM --> Output)
- AWS
- Google Cloud (my preference)
    - Use Mistral through Vertex AI
    - Create a cloud function that can be called by it's endpoint (this is like your own API)
        * Prompt engineering
        * Embeddings work + when to embed...
    - Store the vector database in google cloud storage
    - Either
        * Build own interface (REACT + great UI/UX)
        * Call the endpoint to your GCloud function and ta da :)
    - OR
        * Create some sort of add-on on your chosen platform
        * Google excel, sheets, etc.
- Some other

# Deploying traditional models (Data --> Solvable task - regression or classification, for example --> accurate answer)
- Example: Iris dataset. Input ['1.56', '2.45', '7.5', '1.2'] for ['sepal length', 'sepal width', 'petal lenght', 'petal width']
    * Ask user to input specific metrics, not nuanced prompt.
- Output the most correct answer based on our model's learned relationship.
- Google Cloud
    - Interface asking specific metrics/photo/audio
        * Classify dog/cat/zebra/etc.
    - Gcloud function specifying logic --> will use the model and pass in the input to get an output
    - return output when function endpoint is called
    - display answer to user


Canva --> stunning!